In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [6]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.


In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições estiveram abertas até o dia 11 de maio de 2025.


In [7]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura']
Páginas utilizadas na resposta: starten.tech, youtube.com, alura.com.br



In [8]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [22]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [23]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [24]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [25]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Atue como um Líder experiente em Security Awareness. Sua tarefa é realizar uma busca
        abrangente no Google Search pelas últimas notícias sobre golpes, fraudes e ataques de
        cibersegurança e segurança da informação.

        Você deve:
      - Verificar um máximo de [Número] resultados de busca. (Sugestão inicial: 10)
      - Concentrar-se em notícias publicadas nos últimos 45 dias (ou seja, a partir de aproximadamente 2 de abril de 2025 até hoje, 17 de maio de 2025).
      - Priorizar fontes confiáveis e relevantes na área de segurança da informação.
      - Fornecer um resumo conciso de cada notícia encontrada, incluindo o título, a fonte e a data de publicação.
      - Listar os links diretos para cada uma das notícias encontradas.

      O objetivo é fornecer ao próximo agente uma lista de notícias relevantes e atuais para análise.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [18]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em análise de conteúdo de Security Awareness. Dada a seguinte lista de notícias (fornecida pelo Buscador de Notícias),
        sua tarefa é selecionar a notícia mais relevante e impactante para a criação de um post de conscientização para os colaboradores de uma empresa.

        Considere os seguintes critérios para a seleção:
        - **Atualidade:** A notícia deve ser recente e refletir ameaças ativas no cenário atual.
        - **Impacto Potencial:** A notícia deve abordar um golpe, fraude ou ataque que possa afetar um número significativo de pessoas ou causar danos relevantes.
        - **Clareza e Compreensão:** A notícia deve ser clara e fácil de entender, mesmo para pessoas sem conhecimento técnico em segurança da informação.
        - **Relevância para o público:** A notícia deve ser algo que os colaboradores da empresa provavelmente encontrarão ou precisarão estar cientes.

        Sua saída deve ser:
        - O título da notícia selecionada.
        - O link direto para a notícia selecionada.
        - Um breve parágrafo explicando o motivo da sua seleção, destacando sua relevância e potencial impacto para os colaboradores.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [19]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um redator experiente em comunicação de Security Awareness, com foco em criar mensagens claras, objetivas e
            impactantes para colaboradores de uma empresa. Com base na seguinte notícia selecionada (fornecida pelo Selecionador de Notícias),
            sua tarefa é criar um post conciso e informativo, seguindo os seguintes tópicos:

          - **Título:** Crie um título chamativo e direto, que capture a atenção do leitor e resuma o tema principal da notícia.
          - **Introdução Sucinta:** Escreva um breve parágrafo (máximo 3-4 frases) introduzindo o golpe, fraude ou ataque de forma
          clara e direta, explicando o que aconteceu.
          - **Riscos:** Detalhe os principais riscos e consequências que os colaboradores podem enfrentar caso caiam nesse tipo de golpe ou ataque.
          Seja específico sobre os impactos potenciais (perda de dados, prejuízo financeiro, acesso indevido a contas, etc.).
          - **Como se Proteger:** Forneça dicas práticas e acionáveis sobre como os colaboradores podem se proteger contra esse tipo de ameaça.
          Use linguagem clara e direta, com passos simples e fáceis de seguir.

          Seu objetivo é criar um post que informe, alerte e oriente os colaboradores sobre como se manterem seguros.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [20]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um revisor de conteúdo de Security Awareness com um olhar crítico e foco na clareza e relevância da informação.
            Sua tarefa é revisar o post gerado pelo Redator de Comunicação, com base na notícia original (fornecida pelo Selecionador de Notícias).

            Você deve avaliar se o post:
            - **Faz sentido:** As informações apresentadas são coerentes e alinhadas com a notícia original?
            - **É preciso:** Os detalhes do golpe, fraude ou ataque estão corretos?
            - **É claro e objetivo:** A linguagem utilizada é fácil de entender por todos os colaboradores?
            - **O título é adequado:** O título captura a essência da notícia e é chamativo?
            - **A introdução é eficaz:** A introdução contextualiza o assunto de forma clara e concisa?
            - **Os riscos estão bem definidos:** Os potenciais impactos para os colaboradores são claros e relevantes?
            - **As dicas de proteção são práticas e acionáveis:** Os colaboradores saberão o que fazer para se proteger?
            - **O tom é adequado:** O tom da mensagem é informativo e de alerta, sem gerar pânico desnecessário?

            Se necessário, sugira melhorias ou correções para o post, visando garantir a máxima clareza e impacto da comunicação.
            Sua saída deve ser um feedback detalhado sobre a qualidade do post e, se aplicável, sugestões de melhoria.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [26]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: IA
Maravilha! Vamos então criar o post sobre novidades em IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Ok, vou realizar a busca no Google pelas últimas notícias sobre golpes, fraudes e ataques de cibersegurança e segurança da informação relacionados a IA, com foco nos últimos 45 dias e priorizando fontes confiáveis.
> 
> 
> Com certeza! Aqui está um resumo das notícias mais recentes sobre golpes, fraudes e ataques de cibersegurança e segurança da informação relacionados à IA, com foco nos últimos 45 dias:
> 
> **1. Fraude impulsionada por IA: A nova ameaça para empresas**
> 
> *   **Fonte:** WeLiveSecurity (ESET)
> *   **Data:** 12 de março de 2025
> *   **Resumo:** A inteligência artificial (IA) está sendo utilizada por cibercriminosos para criar golpes digitais mais sofisticados, incluindo a criação de funcionários falsos com currículos e documentos falsificados, e o uso de deepfakes em fraudes Business Email Compromise (BEC).
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF882Jmu5hlbG_2WuRLN_YYIfQMzL04uPdYh-gJau_uDrjj0bKJrhDw-ybhHbQaq-pyg_XJkYvoDpTeqG4S1u5q9xAiNX1nexpQ0g0D8br5JWmxVJDRBwKbZofiOOT-iJIlmaaBWvHyyPwr-WBJhdAQV-JSl_GDKUXo4vhrs_YXI8u-q5Xk33_agEJ4klhgVvQ6GazAaVYIt0wwEwLZQoUsX4mTtvsTZ_WHiQqg](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF882Jmu5hlbG_2WuRLN_YYIfQMzL04uPdYh-gJau_uDrjj0bKJrhDw-ybhHbQaq-pyg_XJkYvoDpTeqG4S1u5q9xAiNX1nexpQ0g0D8br5JWmxVJDRBwKbZofiOOT-iJIlmaaBWvHyyPwr-WBJhdAQV-JSl_GDKUXo4vhrs_YXI8u-q5Xk33_agEJ4klhgVvQ6GazAaVYIt0wwEwLZQoUsX4mTtvsTZ_WHiQqg)
> 
> **2. Avanços e riscos da Inteligência Artificial na Cibersegurança**
> 
> *   **Fonte:** Protiviti
> *   **Data:** 1 de abril de 2025
> *   **Resumo:** A IA está sendo usada tanto para fortalecer a cibersegurança quanto para potencializar ataques cibernéticos. Algoritmos de deepfake e técnicas de processamento de linguagem natural (NLP) estão sendo usados para criar e-mails de phishing personalizados e ataques de força bruta mais eficientes.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGKzyDhWCa7l84r41rMMQZSVsujF2cnxdrQT-W8tzop2CyfmCzG4pu5GQ_P5M06inENGFPLsQh_fbJCL47aMlL6FEVPthO8SazGtHwmJdKqkBXVYuKIyrADARMOVDAE7HzF73wCjYXbvAGMnrYxQov9TNrrtgWeNlHQHuAkfqMeNZ2qHhq-ZJanoWA67UykhltHIiE5n4uGUEEEsLLy6rtzdWfioQ==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGKzyDhWCa7l84r41rMMQZSVsujF2cnxdrQT-W8tzop2CyfmCzG4pu5GQ_P5M06inENGFPLsQh_fbJCL47aMlL6FEVPthO8SazGtHwmJdKqkBXVYuKIyrADARMOVDAE7HzF73wCjYXbvAGMnrYxQov9TNrrtgWeNlHQHuAkfqMeNZ2qHhq-ZJanoWA67UykhltHIiE5n4uGUEEEsLLy6rtzdWfioQ==)
> 
> **3. Cisco: 77% das empresas no Brasil sofrem ataques com IA, mas 40% subestimam riscos**
> 
> *   **Fonte:** Portal Contábil SC
> *   **Data:** 9 de maio de 2025
> *   **Resumo:** Apesar de 93% das empresas brasileiras usarem IA para detectar ameaças, 77% sofreram incidentes relacionados à tecnologia no último ano, e 40% dos gestores subestimam os riscos. Muitos funcionários usam ferramentas de IA generativa sem monitoramento, abrindo portas para vazamentos de dados.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG9o1LqdJ20QiAkzDXtHuC3AWaM16tlYv72W4jEnhb-dG22JvZdmwa_5TyDm0CIXwXba2EU20URBQF5F4DfYlFQ6HLl6y2btafs3vCLinMOeZNcMcrf6nuO08lhu60CFWEbEpJJRnk3AWkru4MCI0KfHvvmpXL4fpNzDXBJUxx8zLC2phC6fK5Vky5P0fyIOWZGW44EbceG3D1pUmDq5oYWOTd-ojJole5USOsoXVPUBd8p7Q==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXG9o1LqdJ20QiAkzDXtHuC3AWaM16tlYv72W4jEnhb-dG22JvZdmwa_5TyDm0CIXwXba2EU20URBQF5F4DfYlFQ6HLl6y2btafs3vCLinMOeZNcMcrf6nuO08lhu60CFWEbEpJJRnk3AWkru4MCI0KfHvvmpXL4fpNzDXBJUxx8zLC2phC6fK5Vky5P0fyIOWZGW44EbceG3D1pUmDq5oYWOTd-ojJole5USOsoXVPUBd8p7Q==)
> 
> **4. Fraudes online no Brasil aumentam com uso de inteligência artificial**
> 
> *   **Fonte:** Notícias R7
> *   **Data:** 17 de maio de 2025
> *   **Resumo:** Criminosos estão utilizando IA para aprimorar fraudes online no Brasil, resultando em aproximadamente 26 tentativas de golpe por minuto.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHlusMYlacmoax4eZcGtdpb6_h_i-whoA9o9Se1VpGMVOs0VVkcE1-Lpp8bGEh4NGbU7tC8NIP5glU4zCcK8rWVEGJfogMia1G6ko5G5tL-oD8tD8N_uY7AlUM5TSaxsA-BPnoGsU5l7OYTwQ3w994lGAE7rmyTZZK1KiHWSVr5yzHkwwPfE0PodldJo2XodvgMPBs2BSVXqzg56eB33dv7Y_BguT0_p1S9yQ==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHlusMYlacmoax4eZcGtdpb6_h_i-whoA9o9Se1VpGMVOs0VVkcE1-Lpp8bGEh4NGbU7tC8NIP5glU4zCcK8rWVEGJfogMia1G6ko5G5tL-oD8tD8N_uY7AlUM5TSaxsA-BPnoGsU5l7OYTwQ3w994lGAE7rmyTZZK1KiHWSVr5yzHkwwPfE0PodldJo2XodvgMPBs2BSVXqzg56eB33dv7Y_BguT0_p1S9yQ==)
> 
> **5. Inteligência Artificial fortalece segurança do e-commerce contra fraudes**
> 
> *   **Fonte:** E-Commerce Brasil
> *   **Data:** 30 de abril de 2025
> *   **Resumo:** A IA está sendo usada para analisar grandes volumes de dados em tempo real, identificar padrões suspeitos e bloquear transações fraudulentas em e-commerces, além de reduzir falsos positivos.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEh06J8z523_IMN0I3Ehq4c-hD49frLPFJpzcGNX-Byl5yrolJAdhvpTrLo02JoP4qHJb-nPWEMYvsJglPecHscXgtbITEl5wQLNBqOnipsPm0ClIR0l9RKleYvw5sa10kRzu-lK77FmBzxwMzx396hGhwmAgf-MGKsgtcsvV5W3twpUjI9aApDjqQXRdJRsy40u891ZRj5VUA9gvjv7taI2EwSes9P1jHN0-3br8GM](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEh06J8z523_IMN0I3Ehq4c-hD49frLPFJpzcGNX-Byl5yrolJAdhvpTrLo02JoP4qHJb-nPWEMYvsJglPecHscXgtbITEl5wQLNBqOnipsPm0ClIR0l9RKleYvw5sa10kRzu-lK77FmBzxwMzx396hGhwmAgf-MGKsgtcsvV5W3twpUjI9aApDjqQXRdJRsy40u891ZRj5VUA9gvjv7taI2EwSes9P1jHN0-3br8GM)
> 
> **6. Google passa a usar IA para identificar sites fraudulentos no Chrome**
> 
> *   **Fonte:** CNN Brasil
> *   **Data:** 8 de maio de 2025
> *   **Resumo:** O Google está usando uma versão de seu modelo de IA Gemini para detectar e alertar sobre golpes de "suporte técnico" no Chrome, Busca e Android.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFjYgUY85n0irQU7MtCbinch4eyHZ2KgmW6VGIWETN17kOwqwB9h9binzgDIQQ4W03Seu2nXZAC0rwhn5h1o1sYwlr6uUsu5ELSSuWdkL-0pciljbbSt5Bq7LwbH8SYY5fOmmxxt1PI09MR7VUGHZNIGqQZt-d256tYBktpvnAixJXuoG3OLQffzvD2u0QFxe6t40Y4S-Gd84crIC5R4ysowaZ5I3rWqnU=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFjYgUY85n0irQU7MtCbinch4eyHZ2KgmW6VGIWETN17kOwqwB9h9binzgDIQQ4W03Seu2nXZAC0rwhn5h1o1sYwlr6uUsu5ELSSuWdkL-0pciljbbSt5Bq7LwbH8SYY5fOmmxxt1PI09MR7VUGHZNIGqQZt-d256tYBktpvnAixJXuoG3OLQffzvD2u0QFxe6t40Y4S-Gd84crIC5R4ysowaZ5I3rWqnU=)
> 
> **7. Cibersegurança em tempos de Inteligência Artificial**
> 
> *   **Fonte:** A Voz da Indústria
> *   **Data:** 29 de abril de 2025
> *   **Resumo:** A IA pode ser usada tanto para reforçar a cibersegurança quanto para amplificar ataques cibernéticos, como phishing e malware. É fundamental que as empresas invistam em proteções digitais reforçadas e utilizem a IA com responsabilidade.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGz9QwtN7YUTDSEO_d8UCPCnp7Gzn1R-_OI-1wbU5q6dJPvccoW7U5ovpLzQqSAiT9u4IkzTKCf2sTwOhY6jaBtUR1ehTFVpsWi_Y6mx6Lj41uhKzPe6Q62F1B0OpKdiH6GP91my_O9W3kqgE2l2UvecFJjwRL0OjRUajbni33BSbrQhqqO1Tdg3R7-wkBjPHDadNy5HA==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGz9QwtN7YUTDSEO_d8UCPCnp7Gzn1R-_OI-1wbU5q6dJPvccoW7U5ovpLzQqSAiT9u4IkzTKCf2sTwOhY6jaBtUR1ehTFVpsWi_Y6mx6Lj41uhKzPe6Q62F1B0OpKdiH6GP91my_O9W3kqgE2l2UvecFJjwRL0OjRUajbni33BSbrQhqqO1Tdg3R7-wkBjPHDadNy5HA==)
> 
> **8. Golpes virtuais aumentam e não fazem distinção de idade**
> 
> *   **Fonte:** COAD
> *   **Data:** 15 de abril de 2025
> *   **Resumo:** Jovens entre 16 e 29 anos são as maiores vítimas de golpes virtuais, mas a IA está tornando mais difícil reconhecer fraudes, mesmo para pessoas com menos familiaridade com a tecnologia.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGEihvu6XKodaP-N55cfvaVPDBPMiA2tM1S-AqkwPp8DXN4O88cJyKnE3eG836OSYu629wSLN6ixv50odvulJvM2frjCAWcRs9a67e_e0VoOvmCbFWgxphvQsPfcNBepJhibK1IkK16uQSgz_tSLhL1du1VnTLsxuMcR7ZozkK0MvJhgnMwVeXzNwuFwlT58dtgczSOVvG8UtwUC87swYl90iWvsseF](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGEihvu6XKodaP-N55cfvaVPDBPMiA2tM1S-AqkwPp8DXN4O88cJyKnE3eG836OSYu629wSLN6ixv50odvulJvM2frjCAWcRs9a67e_e0VoOvmCbFWgxphvQsPfcNBepJhibK1IkK16uQSgz_tSLhL1du1VnTLsxuMcR7ZozkK0MvJhgnMwVeXzNwuFwlT58dtgczSOVvG8UtwUC87swYl90iWvsseF)
> 
> **9. Em golpe da ração, criminosos pedem à IA mensagem com “tom profissional“**
> 
> *   **Fonte:** CNN Brasil
> *   **Data:** 17 de maio de 2025
> *   **Resumo:** Golpistas estão usando ferramentas de inteligência artificial, como o ChatGPT, para elaborar mensagens mais convincentes e personalizadas.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEKb1tMPX-otwcihJRjO1GYhplRt3lCBNOjclDEiY_2ysiNxKPCiPOVxxQBsPPADiO3LYp2_XFi2GEqqcOtZQ7uFQsdtt9_flBc4B_9SqsHPtdIuxqiYmjtgcJagy0bB2QP0z1qE8yWUOw5JbJTfF5L4jvIw9vhIrAZsmhi8OboCGswp4rrZKigYyDPJQnz3rM5SoAT5NmVqZjvuTzTudvPGly03zdpHNG0lfNsYuk=](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEKb1tMPX-otwcihJRjO1GYhplRt3lCBNOjclDEiY_2ysiNxKPCiPOVxxQBsPPADiO3LYp2_XFi2GEqqcOtZQ7uFQsdtt9_flBc4B_9SqsHPtdIuxqiYmjtgcJagy0bB2QP0z1qE8yWUOw5JbJTfF5L4jvIw9vhIrAZsmhi8OboCGswp4rrZKigYyDPJQnz3rM5SoAT5NmVqZjvuTzTudvPGly03zdpHNG0lfNsYuk=)
> 
> **10. Uso da Inteligência Artificial na cibersegurança é a tônica da RSA Conference 2025, maior evento mundial do setor**
> 
> *   **Fonte:** Revista LYNX
> *   **Data:** 15 de maio de 2025
> *   **Resumo:** O uso da IA nas ferramentas de cibersegurança foi o tema central da RSA Conference 2025. Praticamente todos os players apresentaram soluções com essa tecnologia, com o intuito de acelerar os processos e ganhar velocidade e maturidade na área de segurança.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE14IVI6NKjq-fGsSuT_VPVdmnolQavSgoRPdIRnfxG_b0mvxZH_ZybMAA23eQsEWNNLfBVBWE8HiJQkCnb9dquwiyrUj3U0x4YXxnNBh8DcTIJUha1PVDRfI1ryMUCzD0q0zEtEtMZt3ad4DYXXz8h4D11dtPPk7XgkKF5Q0ieYxHnCRgQ7JBD](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXE14IVI6NKjq-fGsSuT_VPVdmnolQavSgoRPdIRnfxG_b0mvxZH_ZybMAA23eQsEWNNLfBVBWE8HiJQkCnb9dquwiyrUj3U0x4YXxnNBh8DcTIJUha1PVDRfI1ryMUCzD0q0zEtEtMZt3ad4DYXXz8h4D11dtPPk7XgkKF5Q0ieYxHnCRgQ7JBD)
> 
> **11. IA na defesa e no ataque: a nova ordem da cibersegurança, segundo a CrowdStrike**
> 
> *   **Fonte:** Exame
> *   **Data:** 7 de março de 2025
> *   **Resumo:** As empresas estão acelerando o uso de IA para proteção, mas, ao mesmo tempo, os ataques se tornam mais sofisticados. Com IA, um invasor pode criar vulnerabilidades zero-day a cada momento.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEcN4xk0DwgSaBceXy7mE9A9Vqbva9Sb-Y_ZsekCr8S0mReTKfVlZrKRjnltW3XLwUgN4lPp6NQKHNMCd3ruLVW5AbJe71mLnGbLTZASyf25mJpcBeeyrQdNREtgvO4RYPYllJngEx58CchdAKw4SeVJVR_Cfrg1cjwyc35vBnIYx9HXb8Kus-2xxlOdOHCms4GXaBb7_KJl3mpFyWzzQm6OXb4QM163-vozEBZDLuW182amQ==](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEcN4xk0DwgSaBceXy7mE9A9Vqbva9Sb-Y_ZsekCr8S0mReTKfVlZrKRjnltW3XLwUgN4lPp6NQKHNMCd3ruLVW5AbJe71mLnGbLTZASyf25mJpcBeeyrQdNREtgvO4RYPYllJngEx58CchdAKw4SeVJVR_Cfrg1cjwyc35vBnIYx9HXb8Kus-2xxlOdOHCms4GXaBb7_KJl3mpFyWzzQm6OXb4QM163-vozEBZDLuW182amQ==)
> 
> **12. Cyber Signals 9ª Edição | Enganação impulsionada por IA: Ameaças emergentes de fraude e contramedidas**
> 
> *   **Fonte:** Microsoft News
> *   **Data:** 16 de abril de 2025
> *   **Resumo:** A IA está baixando a barreira técnica para agentes de fraudes e crimes cibernéticos, tornando mais fácil e barato gerar conteúdo convincente para ataques. Golpistas estão usando deepfakes, clonagem de voz, e-mails de phishing e sites falsos para parecer legítimos em maior escala.
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGRaZjTmQ9MUiRRCFbeOivynOItrUm9LqiDI45xggTQSKsZqEaUXpyomBYrRGv33Q9vO3CEXSXmkH3NhX-9ea4ALivrLBXHaYnCon1oz7duShy0B9WJvwvtTkR1Zlqqd8uCdC8OyqjRMWBzDMvR-gzZ6KLA4ClNDeRxKivE_gktv8rTsGlQe1idasOWmGQmWoHM9dDvszuCWa90U3WuEPNfl47nuZXoGOYAjl8Zrvalp3ic-G_k-SsP4JtIjvk7zPPgnTMq0LwcuVBidw6461Ffu4W1osoHJF-MOsh-g_XDqkNzk8a9](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXGRaZjTmQ9MUiRRCFbeOivynOItrUm9LqiDI45xggTQSKsZqEaUXpyomBYrRGv33Q9vO3CEXSXmkH3NhX-9ea4ALivrLBXHaYnCon1oz7duShy0B9WJvwvtTkR1Zlqqd8uCdC8OyqjRMWBzDMvR-gzZ6KLA4ClNDeRxKivE_gktv8rTsGlQe1idasOWmGQmWoHM9dDvszuCWa90U3WuEPNfl47nuZXoGOYAjl8Zrvalp3ic-G_k-SsP4JtIjvk7zPPgnTMq0LwcuVBidw6461Ffu4W1osoHJF-MOsh-g_XDqkNzk8a9)
> 
> Espero que isso seja útil!
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Aqui está a notícia mais relevante para um post de conscientização, baseada nos critérios fornecidos:
> 
> *   **Título:** Fraude impulsionada por IA: A nova ameaça para empresas
> *   **Link:** [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF882Jmu5hlbG_2WuRLN_YYIfQMzL04uPdYh-gJau_uDrjj0bKJrhDw-ybhHbQaq-pyg_XJkYvoDpTeqG4S1u5q9xAiNX1nexpQ0g0D8br5JWmxVJDRBwKbZofiOOT-iJIlmaaBWvHyyPwr-WBJhdAQV-JSl_GDKUXo4vhrs_YXI8u-q5Xk33_agEJ4klhgVvQ6GazAaVYIt0wwEwLZQoUsX4mTtvsTZ_WHiQqg](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXF882Jmu5hlbG_2WuRLN_YYIfQMzL04uPdYh-gJau_uDrjj0bKJrhDw-ybhHbQaq-pyg_XJkYvoDpTeqG4S1u5q9xAiNX1nexpQ0g0D8br5JWmxVJDRBwKbZofiOOT-iJIlmaaBWvHyyPwr-WBJhdAQV-JSl_GDKUXo4vhrs_YXI8u-q5Xk33_agEJ4klhgVvQ6GazAaVYIt0wwEwLZQoUsX4mTtvsTZ_WHiQqg)
> 
> **Motivo da seleção:**
> 
> Esta notícia é de grande relevância e impacto para os colaboradores de uma empresa porque destaca como a inteligência artificial está sendo utilizada para sofisticar golpes digitais. A criação de funcionários falsos com currículos e documentos forjados e o uso de deepfakes em fraudes BEC podem enganar até mesmo os funcionários mais experientes. Alertar os colaboradores sobre essas novas táticas de fraude impulsionadas por IA pode aumentar a vigilância e reduzir o risco de a empresa ser vítima desses ataques. Além disso, a notícia é recente e de uma fonte confiável (ESET/WeLiveSecurity).
> 


--------------------------------------------------------------


Exception in thread Thread-28 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 126, in _invoke_run_async
    async for event in self.run_async(
  File "/u


--- 📝 Resultado do Agente 3 (Redator) ---



--------------------------------------------------------------
